In [1]:
import tensorflow as tf 

In [22]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self, n_units,**kwargs):
        self.eps = 0.001
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.alphas = self.add_weight(shape=(input_shape[-1:]), initializer='ones',dtype=tf.float32)
        self.betas = self.add_weight(shape=(input_shape[-1:]), initializer='zeros',dtype=tf.float32)
    
    def call(self, X):
        mean, var = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alphas * (X-mean)/(tf.sqrt(var)+self.eps) + self.betas


Then the function should compute and return α ⊗ (X – μ)/(σ + ε)
\+ β, where ⊗ represents itemwise multiplication (*) and ε is a smoothing term
(a small constant to avoid division by zero, e.g., 0.001).

In [23]:
(X_train, y_train) , (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [24]:
X_train = X_train.reshape(-1, 28*28,)#.shape, y_train.shape
X_test = X_test.reshape(-1, 28*28,)#.shape, y_train.shape

In [26]:
inputs = tf.keras.layers.Input(shape=(784,))
z = MyLayer(30)(inputs)
# outputs = tf.keras.layers.Dense(10, activation='softmax')(z)
model = tf.keras.Model(inputs=inputs, outputs=z)

In [37]:
model(X_train)

<tf.Tensor: shape=(60000, 784), dtype=float32, numpy=
array([[-0.4407846 , -0.4407846 , -0.4407846 , ..., -0.4407846 ,
        -0.4407846 , -0.4407846 ],
       [-0.47279605, -0.47279605, -0.47279605, ..., -0.47279605,
        -0.47279605, -0.47279605],
       [-0.37815583, -0.37815583, -0.37815583, ..., -0.37815583,
        -0.37815583, -0.37815583],
       ...,
       [-0.39481747, -0.39481747, -0.39481747, ..., -0.39481747,
        -0.39481747, -0.39481747],
       [-0.39231744, -0.39231744, -0.39231744, ..., -0.39231744,
        -0.39231744, -0.39231744],
       [-0.3949141 , -0.3949141 , -0.3949141 , ..., -0.3949141 ,
        -0.3949141 , -0.3949141 ]], dtype=float32)>

In [39]:
tf.keras.backend.clear_session()
inputs_compare = tf.keras.layers.Input(shape=(784,))
z_compare = tf.keras.layers.LayerNormalization()(inputs_compare)
model_compare = tf.keras.Model(inputs = inputs_compare, outputs=z_compare)

In [40]:
model_compare(X_train)

<tf.Tensor: shape=(60000, 784), dtype=float32, numpy=
array([[-0.4407901 , -0.4407901 , -0.4407901 , ..., -0.4407901 ,
        -0.4407901 , -0.4407901 ],
       [-0.47280166, -0.47280166, -0.47280166, ..., -0.47280166,
        -0.47280166, -0.47280166],
       [-0.3781615 , -0.3781615 , -0.3781615 , ..., -0.3781615 ,
        -0.3781615 , -0.3781615 ],
       ...,
       [-0.39482293, -0.39482293, -0.39482293, ..., -0.39482293,
        -0.39482293, -0.39482293],
       [-0.3923233 , -0.3923233 , -0.3923233 , ..., -0.3923233 ,
        -0.3923233 , -0.3923233 ],
       [-0.39491987, -0.39491987, -0.39491987, ..., -0.39491987,
        -0.39491987, -0.39491987]], dtype=float32)>